# SageMaker Example

## 2. Build the container

demo codes are in `app/`
build and push the docker with following commands:

In [6]:
!pip install  -U sagemaker boto3

^C
ERROR: Operation cancelled by user


In [ ]:
!bash build_and_push_sglang.sh

## 3. Deploy on SageMaker

define the model and deploy on SageMaker


### 3.1 Init SageMaker session

In [30]:
# !pip install boto3 sagemaker transformers
import re
import json
import os,dotenv
import boto3
import sagemaker
from sagemaker import Model


dotenv.load_dotenv()
print(os.environ)

boto_sess = boto3.Session(
    region_name='us-east-1'
)

sess = sagemaker.session.Session(boto_session=boto_sess)
# role = sagemaker.get_execution_role()
role = os.environ.get('role')

environ({'USER': 'ubuntu', 'SSH_CLIENT': '72.21.198.67 31229 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'HOME': '/home/ubuntu', 'SSL_CERT_FILE': '/usr/lib/ssl/cert.pem', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'LOGNAME': 'ubuntu', '_': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '35677', 'VSCODE_CLI_REQUIRE_TOKEN': '14293638-00a0-43f8-a1f9-97ee8de4709f', 'PATH': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin:/home/ubuntu/.vscode-server/cli/servers/Stable-6609ac3d66f4eade5cf376d1cb76f13985724bcb/server/bin/remote-cli:/home/ubuntu/.local/bin:/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin:/home/ubuntu/workspace/llm_model_hub/miniconda3/condabin:/home/ubuntu/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT_FOLDER': '/home/ubuntu/.vscode-server', 'XDG_RUNTIME_DIR': '/run/user/1000', 'S

[03/08/25 06:29:28] INFO     Found credentials from IAM Role: admin_role_for_workshop           ]8;id=209435;file:///home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=866294;file:///home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore/credentials.py#1132\1132]8;;\

In [ ]:
# role

### 3.2 Prepare model file

#### Option 2: deploy vllm by model_id

In [ ]:
!tar czvf model.tar.gz model_tar/

In [31]:


s3_code_prefix = f"sagemaker_endpoint/sglang/"
bucket = sess.default_bucket() 
code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-434444145045/sagemaker_endpoint/sglang//model.tar.gz


### 3.3 Deploy model

### test deployment from s3

In [33]:
import sagemaker
import boto3
sm_client = boto3.client(service_name="sagemaker")

# base_name = sagemaker.utils.name_from_base("sagemaker")
base_name = "Qwen2-5-VL-3B-Instruct--sglang-endpoint"
endpoint_name = base_name+"-endpoint"
endpoint_config_name =  base_name+"-config"
model_name = base_name +"-model"

inference_component_name = base_name+"-component"


## Setup your SageMaker Real-time Endpoint
### Create a SageMaker endpoint configuration

We begin by creating the endpoint configuration and set MinInstanceCount to 0. This allows the endpoint to scale in all the way down to zero instances when not in use.


In [46]:
instance_type = "ml.g5.2xlarge"
max_instance_count = 2

resp = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 1200,
            "ContainerStartupHealthCheckTimeoutInSeconds": 1200,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": 1,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)

### Create the SageMaker endpoint

Next, we create our endpoint using the above endpoint config


In [47]:
sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)
print(f"Endpoint name: {endpoint_name}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 sm_client.create_endpoint(                                                                   │
│   2 │   EndpointName=endpoint_name,                                                              │
│   3 │   EndpointConfigName=endpoint_config_name,                                                 │
│   4 )                                                                                            │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /client.py:570 in _api_call                                                                      │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /context.py:124 in wrapper                                                                       │
│                                                                                                  │
│   121 │   │   │   with start_as_current_context():                                               │
│   122 │   │   │   │   if hook:                                                                   │
│   123 │   │   │   │   │   hook()                                                                 │
│ ❱ 124 │   │   │   │   return func(*args, **kwargs)                                               │
│   125 │   │                                                                                      │
│   126 │   │   return wrapper                                                                     │
│   127                                                                                            │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /client.py:1031 in _make_api_call                                                                │
│                                                                                                  │
│   1028 │   │   │   │   "Code"                                                                    │
│   1029 │   │   │   )                                                                             │
│   1030 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1031 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1032 │   │   else:                                                                             │
│   1033 │   │   │   return parsed_response                                                        │
│   1034                                                     

### We wait for our endpoint to go InService. This step can take ~3 mins

In [50]:
import time
import sys
# Let's see how much it takes
start_time = time.time()

while True:
    desc = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = desc["EndpointStatus"]
    print(desc)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

{'EndpointName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-endpoint', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint/Qwen2-5-VL-3B-Instruct--sglang-endpoint-endpoint', 'EndpointConfigName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-config', 'ProductionVariants': [{'VariantName': 'AllTraffic', 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1, 'ManagedInstanceScaling': {'Status': 'ENABLED', 'MinInstanceCount': 1, 'MaxInstanceCount': 2}, 'RoutingConfig': {'RoutingStrategy': 'LEAST_OUTSTANDING_REQUESTS'}}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2025, 3, 8, 6, 31, 11, 396000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 3, 8, 6, 33, 30, 246000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '2c41c27a-4254-4632-9379-e7f1feea2786', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2c41c27a-4254-4632-9379-e7f1feea2786', 'content-type': 'application/x-amz-json-1.1', 'content-length': '604', 'date': 'Sat, 08 Mar 20

In [63]:
sm_client.describe_endpoint(EndpointName=endpoint_name)

{'EndpointName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint/Qwen2-5-VL-3B-Instruct--sglang-endpoint-endpoint',
 'EndpointConfigName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-config',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1,
   'ManagedInstanceScaling': {'Status': 'ENABLED',
    'MinInstanceCount': 1,
    'MaxInstanceCount': 2},
   'RoutingConfig': {'RoutingStrategy': 'LEAST_OUTSTANDING_REQUESTS'}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2025, 3, 8, 6, 31, 11, 396000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 3, 8, 6, 33, 30, 246000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '7e605826-93af-47d2-b1e2-64a97528a95e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7e605826-93af-47d2-b1e2-64a97528a95e',
   'content-type': 'application/x-amz-json-1.1',
   'content-le

### Create a SageMaker model



(llama3-8b-scale-to-zero-autoscaling)[https://github.com/aws-samples/sagemaker-genai-hosting-examples/blob/main/scale-to-zero-endpoint/llama3-8b-scale-to-zero-autoscaling.ipynb]

In [52]:

CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/sglang:v0.4.3.post2-cu124'
# CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/vllm:v0.7.2'
model_path = "s3://sagemaker-us-east-1-434444145045/Qwen2-5-7B-Instruct/7c7077a7bda14263a190a342e3817a7d/finetuned_model/"
model_id = 'Qwen/Qwen2.5-7B-Instruct'
sm_client = boto3.client(service_name="sagemaker")


env={
    "HF_MODEL_ID": model_id,
    "S3_MODEL_PATH":model_path,
}
container_config = {
    'Image': CONTAINER,
    'ModelDataUrl': code_artifact,
    'Environment': env
}

print(model_name)
print(endpoint_name)
print(inference_component_name)
print(endpoint_config_name)

Qwen2-5-VL-3B-Instruct--sglang-endpoint-model
Qwen2-5-VL-3B-Instruct--sglang-endpoint-endpoint
Qwen2-5-VL-3B-Instruct--sglang-endpoint-component
Qwen2-5-VL-3B-Instruct--sglang-endpoint-config


In [53]:

response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer=container_config
)

print(f"Model created: {response['ModelArn']}")

Model created: arn:aws:sagemaker:us-east-1:434444145045:model/Qwen2-5-VL-3B-Instruct--sglang-endpoint-model


### Create the inference component for your endpoint

In [58]:
import time
t1 = time.time()
sm_client.create_inference_component(
    InferenceComponentName=inference_component_name,
    EndpointName=endpoint_name,
    VariantName="AllTraffic",
    Specification={
        "ModelName": model_name,
        "ComputeResourceRequirements": {
		    "NumberOfAcceleratorDevicesRequired": 1, 
			"MinMemoryRequiredInMb": 1024*8
	    }
    },
    RuntimeConfig={"CopyCount": 1},
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 import time                                                                                 │
│    2 t1 = time.time()                                                                            │
│ ❱  3 sm_client.create_inference_component(                                                       │
│    4 │   InferenceComponentName=inference_component_name,                                        │
│    5 │   EndpointName=endpoint_name,                                                             │
│    6 │   VariantName="AllTraffic",                                                               │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /client.py:570 in _api_call                                                                      │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /context.py:124 in wrapper                                                                       │
│                                                                                                  │
│   121 │   │   │   with start_as_current_context():                                               │
│   122 │   │   │   │   if hook:                                                                   │
│   123 │   │   │   │   │   hook()                                                                 │
│ ❱ 124 │   │   │   │   return func(*args, **kwargs)                                               │
│   125 │   │                                                                                      │
│   126 │   │   return wrapper                                                                     │
│   127                                                                                            │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /client.py:1031 in _make_api_call                                                                │
│                                                                                                  │
│   1028 │   │   │   │   "Code"                                                                    │
│   1029 │   │   │   )                                                                             │
│   1030 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1031 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1032 │   │   else:                                       

### Check the status of IC

In [ ]:
def check_inference_component_status(inference_component_name):
    sagemaker_client = boto3.client('sagemaker')
    
    try:
        response = sagemaker_client.describe_inference_component(
            InferenceComponentName=inference_component_name
        )
        print(response)
        status = response['InferenceComponentStatus']
        print(f"Inference Component Status: {status}")
        
        if status == 'InService':
            return True
        elif status in ['Creating', 'Updating']:
            print("Inference component is still being deployed...")
            return False
        else:
            print(f"Inference component deployment failed with status: {status}")
            # You might want to check the FailureReason if available
            if 'FailureReason' in response:
                print(f"Failure reason: {response['FailureReason']}")
            return True
            
    except Exception as e:
        print(f"Error checking inference component status: {e}")
        return True
 

In [66]:
import time
t1 = time.time()
while True:
    if  check_inference_component_status(inference_component_name):
        print(f"Inference Component is ready:{time.time()-t1:.1f} s")
        break
    time.sleep(30)

{'InferenceComponentName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-component', 'InferenceComponentArn': 'arn:aws:sagemaker:us-east-1:434444145045:inference-component/Qwen2-5-VL-3B-Instruct--sglang-endpoint-component', 'EndpointName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-endpoint', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint/qwen2-5-vl-3b-instruct--sglang-endpoint-endpoint', 'VariantName': 'AllTraffic', 'Specification': {'ModelName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-model', 'ComputeResourceRequirements': {'NumberOfAcceleratorDevicesRequired': 1.0, 'MinMemoryRequiredInMb': 8192}}, 'RuntimeConfig': {'DesiredCopyCount': 1, 'CurrentCopyCount': 1}, 'CreationTime': datetime.datetime(2025, 3, 8, 6, 34, 32, 194000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 3, 8, 6, 41, 50, 271000, tzinfo=tzlocal()), 'InferenceComponentStatus': 'InService', 'ResponseMetadata': {'RequestId': '99813b45-7b1e-4787-bdac-03e174e78614', 'HTTPStatusCode': 200, 'HTT

In [67]:
sm_client.describe_inference_component(
            InferenceComponentName=inference_component_name
        )

{'InferenceComponentName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-component',
 'InferenceComponentArn': 'arn:aws:sagemaker:us-east-1:434444145045:inference-component/Qwen2-5-VL-3B-Instruct--sglang-endpoint-component',
 'EndpointName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint/qwen2-5-vl-3b-instruct--sglang-endpoint-endpoint',
 'VariantName': 'AllTraffic',
 'Specification': {'ModelName': 'Qwen2-5-VL-3B-Instruct--sglang-endpoint-model',
  'ComputeResourceRequirements': {'NumberOfAcceleratorDevicesRequired': 1.0,
   'MinMemoryRequiredInMb': 8192}},
 'RuntimeConfig': {'DesiredCopyCount': 1, 'CurrentCopyCount': 1},
 'CreationTime': datetime.datetime(2025, 3, 8, 6, 34, 32, 194000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 3, 8, 6, 41, 50, 271000, tzinfo=tzlocal()),
 'InferenceComponentStatus': 'InService',
 'ResponseMetadata': {'RequestId': 'fdde7c88-eb17-4b7e-8462-5367242db7c8',
  'HTTPStatu

In [65]:

from sagemaker import Predictor
from sagemaker import serializers, deserializers

predictor = Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sess,
        serializer=serializers.JSONSerializer(),
        component_name=inference_component_name
    )

payload = {
"messages": [
{
    "role": "user",
    "content": "who are you"
}
],
"model":"qwen",
"max_tokens": 1024,
"stream": False
}
response = predictor.predict(payload) 
print(response)

b'{"id":"8ad3016e92e84e82871319b5794d5c9c","object":"chat.completion","created":1741416659,"model":"qwen","choices":[{"index":0,"message":{"role":"assistant","content":"Qwen\\nI\'m Qwen, a large language model created by Alibaba Cloud. I can answer a wide variety of questions and even generate human-like text. What would you like to know?","tool_calls":null},"logprobs":null,"finish_reason":"stop","matched_stop":151645}],"usage":{"prompt_tokens":32,"total_tokens":72,"completion_tokens":40,"prompt_tokens_details":null}}'


### 4.3 Register a new autoscaling target
After you create your SageMaker endpoint and inference components, you register a new auto scaling target for Application Auto Scaling. In the following code block, you set MinCapacity to 0, which is required for your endpoint to scale down to zero

https://aws.amazon.com/cn/blogs/machine-learning/unlock-cost-savings-with-the-new-scale-down-to-zero-feature-in-amazon-sagemaker-inference/

https://github.com/aws-samples/sagemaker-genai-hosting-examples/blob/main/scale-to-zero-endpoint/llama3-8b-scale-to-zero-autoscaling.ipynb

In [ ]:
aas_client = sess.boto_session.client("application-autoscaling")
cloudwatch_client = sess.boto_session.client("cloudwatch")

- 这一步必须，先注册resource_id

In [ ]:

# Autoscaling parameters
resource_id = f"inference-component/{inference_component_name}"
service_namespace = "sagemaker"
scalable_dimension = "sagemaker:inference-component:DesiredCopyCount"

min_copy_count = 1
max_copy_count = 3

aas_client.register_scalable_target(
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    MinCapacity=min_copy_count,
    MaxCapacity=max_copy_count,
)

{'ScalableTargetARN': 'arn:aws:application-autoscaling:us-east-1:434444145045:scalable-target/056mf1f8bbf0dbb446fb8d39ab955d2e6f71',
 'ResponseMetadata': {'RequestId': '2d87f492-ae23-4031-91e8-ce74ac73d421',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2d87f492-ae23-4031-91e8-ce74ac73d421',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '131',
   'date': 'Wed, 05 Mar 2025 16:05:38 GMT'},
  'RetryAttempts': 0}}


### Configure Target Tracking Scaling Policy

- Once you have registered your new scalable target, the next step is to define your target tracking policy. In the code example that follows, we set the TargetValue to 5. This setting instructs the auto-scaling system to increase capacity when the number of concurrent requests per model reaches or exceeds 5. Here we are taking advantage of the more granular auto scaling metric PredefinedMetricType: SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution to more accurately monitor and react to changes in inference traffic. Take a look this blog for more information.


In [ ]:
# The policy name for the target traking policy
target_tracking_policy_name = f"Target-tracking-policy-{inference_component_name}"

aas_client.put_scaling_policy(
    PolicyName=target_tracking_policy_name,
    PolicyType="TargetTrackingScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    TargetTrackingScalingPolicyConfiguration={
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution",
        },
        # Low TPS + load TPS
        "TargetValue": 5,  # you need to adjust this value based on your use case
        "ScaleInCooldown": 60,  # default 300
        "ScaleOutCooldown": 60,  # default 300
    },
)

- Application Auto Scaling creates two CloudWatch alarms per scaling target. The first triggers scale-out actions after 30 seconds (using 3 sub-minute data point), while the second triggers scale-in after 15 minutes (using 90 sub-minute data points). The time to trigger the scaling action is usually 1–2 minutes longer than those minutes because it takes time for the endpoint to publish metrics to CloudWatch, and it also takes time for AutoScaling to react.

### Scale out from zero policy (step scaling policy )
To enable your endpoint to scale out from zero instances, do the following:

Configure Step Scaling Policy
Create a step scaling policy that defines when and how to scale out from zero. This policy will add 1 model copy when triggered, enabling SageMaker to provision the instances required to handle incoming requests after being idle. The following shows you how to define a step scaling policy. Here we have configured to scale out from 0 to 1 model copy ("ScalingAdjustment": 1), depending on your use case you can adjust ScalingAdjustment as required.


In [ ]:
#The policy name for the step scaling policy

step_scaling_policy_name = f"Step-scaling-policy-{inference_component_name}"

aas_client.put_scaling_policy(
    PolicyName=step_scaling_policy_name,
    PolicyType="StepScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    StepScalingPolicyConfiguration={
        "AdjustmentType": "ChangeInCapacity",
        "MetricAggregationType": "Maximum",
        "Cooldown": 60,
        "StepAdjustments":
          [
             {
               "MetricIntervalLowerBound": 0,
               "ScalingAdjustment": 1
             }
          ]
    },
)

In [ ]:
resp = aas_client.describe_scaling_policies(
    PolicyNames=[step_scaling_policy_name],
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
)
step_scaling_policy_arn = resp['ScalingPolicies'][0]['PolicyARN']
print(f"step_scaling_policy_arn: {step_scaling_policy_arn}")
print(resp['ScalingPolicies'])

### Create the CloudWatch alarm that will trigger our policy
Finally, create a CloudWatch alarm with the metric NoCapacityInvocationFailures. When triggered, the alarm initiates the previously defined scaling policy. For more information about the NoCapacityInvocationFailures metric, see documentation.

We have also set the following:  

- EvaluationPeriods to 1  
- DatapointsToAlarm to 1  
- ComparisonOperator to GreaterThanOrEqualToThreshold  
- This results in 1 min waiting for the step scaling policy to trigger  



In [ ]:
# The alarm name for the step scaling alarm
step_scaling_alarm_name = f"step-scaling-alarm-scale-to-zero-aas-{inference_component_name}"

cloudwatch_client.put_metric_alarm(
    AlarmName=step_scaling_alarm_name,
    AlarmActions=[step_scaling_policy_arn],  # Replace with your actual ARN
    MetricName='NoCapacityInvocationFailures',
    Namespace='AWS/SageMaker',
    Statistic='Maximum',
    Dimensions=[
        {
            'Name': 'InferenceComponentName',
            'Value': inference_component_name  
        }
    ],
    Period=30, # 定义了 CloudWatch 收集和聚合指标数据的时间间隔，CloudWatch 支持的最小 Period 值通常为 10 或 60 秒，取决于指标类型和监控级别
    EvaluationPeriods=1, #定义了在多少个连续的数据点中需要满足条件才会触发警报，=1 表示只需要评估 1 个时间段（即 30 秒，由 Period 定义）
    DatapointsToAlarm=1, #表示在 EvaluationPeriods 中需要满足阈值条件的数据点数量，=1 表示在评估的 1 个时间段内，只要有 1 个数据点满足条件就触发警报
    Threshold=1, #表示当 NoCapacityInvocationFailures 指标值大于或等于 1 时触发警报
    ComparisonOperator='GreaterThanOrEqualToThreshold',
    TreatMissingData='missing' #缺失的数据点不会触发警报状态的任何变化，notBreaching将缺失的数据点视为"良好"或"未违反阈值"，breaching将缺失的数据点视为"违反阈值"，ignore保持当前警报状态不变，直到有新数据点出现
)

### Test the solution
- When our SageMaker endpoint doesn’t receive requests for 15 minutes, it will automatically scale down to zero the number of model copies
- After 10 additional minutes of inactivity, SageMaker automatically stops all underlying instances of the endpoint, eliminating all associated instance costs.

In [ ]:
import sys
import time
# time.sleep(900)
start_time = time.time()
while True:
    desc = sm_client.describe_inference_component(InferenceComponentName=inference_component_name)
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

desc = sm_client.describe_inference_component(InferenceComponentName=inference_component_name)
print(desc)

Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
Updating
InService

Total time taken: 542.88 seconds (9.05 minutes)
{'InferenceComponentName': 'huolala-distil-7b-920-full-rag-all-thinking-epoch-2-component', 'InferenceComponentArn': 'arn:aws:sagemaker:us-east-1:434444145045:inference-component/huolala-distil-7b-920-full-rag-all-thinking-epoch-2-component', 'EndpointName': 'huolala-distil-7b-920-full-rag-all-thinking-epoch-2-endpoint', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint/huolala-distil-7b-920-full-rag-all-thinking-epoch-2-endpoint', 'VariantName': 'AllTraffic', 'Specification': {'ModelName': 'huolala-distil-7b-920-full-rag-all-thinking-epoch-2-model', 'ComputeResourceRequirements': {'NumberOfAcceleratorDevicesRequired': 1.0, 'MinMemoryRequiredInMb': 8192}}, 'RuntimeConfig': {'DesiredCopyCount': 2, 'CurrentCopyCount': 2}, 'CreationTime': datetime.datetime(2025, 3,

In [ ]:
sm_client.describe_inference_component(InferenceComponentName="Qwen2-5-VL-3B-Instruct-2025-03-07-05-12-sglang-component")

{'InferenceComponentName': 'Qwen2-5-VL-3B-Instruct-2025-03-07-05-12-sglang-component',
 'InferenceComponentArn': 'arn:aws:sagemaker:us-east-1:434444145045:inference-component/Qwen2-5-VL-3B-Instruct-2025-03-07-05-12-sglang-component',
 'EndpointName': 'Qwen2-5-VL-3B-Instruct-2025-03-07-05-12-sglang-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint/qwen2-5-vl-3b-instruct-2025-03-07-05-12-sglang-endpoint',
 'VariantName': 'AllTraffic',
 'Specification': {'ModelName': 'Qwen2-5-VL-3B-Instruct-2025-03-07-05-12-sglang-model',
  'ComputeResourceRequirements': {'NumberOfAcceleratorDevicesRequired': 1.0,
   'MinMemoryRequiredInMb': 8192}},
 'RuntimeConfig': {'DesiredCopyCount': 1, 'CurrentCopyCount': 1},
 'CreationTime': datetime.datetime(2025, 3, 7, 5, 15, 12, 993000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 3, 7, 5, 56, 12, 289000, tzinfo=tzlocal()),
 'InferenceComponentStatus': 'InService',
 'ResponseMetadata': {'RequestId': '4182e685-dbbb-40

### Endpoint's instances scale in to zero
After 10 additional minutes of inactivity, SageMaker automatically terminates all underlying instances of the endpoint, eliminating all associated costs.

- After 10 additional minutes of inactivity, SageMaker automatically stops all underlying instances of the endpoint, eliminating all associated instance costs.

If we try to invoke our endpoint while instances are scaled down to zero, we get a validation error:  
`An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component has no capacity to process this request. ApplicationAutoScaling may be in-progress (if configured) or try to increase the capacity by invoking UpdateInferenceComponentRuntimeConfig API.`

In [ ]:
from sagemaker import Predictor
from sagemaker import serializers

predictor = Predictor(
            endpoint_name="Qwen2-5-VL-3B-Instruct-2025-03-07-05-12-sglang-endpoint",#endpoint_name,
            sagemaker_session=sess,
            serializer=serializers.JSONSerializer(),
            component_name="Qwen2-5-VL-3B-Instruct-2025-03-07-05-12-sglang-component"#inference_component_name
        )

payload = {
    "messages": [
    {
        "role": "user",
        "content": "hello"
    }
    ],
    "model":"qwen",
    "max_tokens": 5,
    "stream": False
}
response = predictor.predict(payload) 
print(response)

b'{"id":"b0fb5986e9de47e3b9f5823a565e8957","object":"chat.completion","created":1741326966,"model":"qwen","choices":[{"index":0,"message":{"role":"assistant","content":"Hello! How can I","tool_calls":null},"logprobs":null,"finish_reason":"length","matched_stop":null}],"usage":{"prompt_tokens":20,"total_tokens":25,"completion_tokens":5,"prompt_tokens_details":null}}'


### 并发测试

In [ ]:
import asyncio
import concurrent.futures
import math
import nest_asyncio

# 允许在Jupyter中嵌套事件循环
nest_asyncio.apply()

def invoke_model(payload):
    try:
        response = predictor.predict(payload) 
    except Exception as e:
        print(e)
        return None
    return response
    
def split_into_groups(lst, n):
    """将列表分成n个大致相等的组"""
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

async def invoke_model_async( payload):
    # 使用线程池执行同步的invoke_model函数
    loop = asyncio.get_event_loop()
    with concurrent.futures.ThreadPoolExecutor() as pool:
        return await loop.run_in_executor(pool, invoke_model,  payload)

async def process_prompt(idx ):
    # print(f'-----------idx:{idx}-------------------')
    payload = {
        "messages": [
        {
            "role": "user",
            "content": "hello,what can you do?"
        }
        ],
        "model":"qwen",
        "max_tokens": 50,
        "stream": False
    }
    return await invoke_model_async(payload)


async def process_group(num_times):
    """处理一组提示"""
    results = []    
    for idx in range(num_times):
        result = await process_prompt(idx)
        results.append(result)
    return results

async def run_parallel_processing(num_times,threads = 8):
    to_eval_list = []
    
    # 为每个组创建一个任务
    tasks = [
        process_group(num_times)
        for group_idx in range(threads)
    ]
    
    # 并行执行所有组任务
    group_results = await asyncio.gather(*tasks)
    
    # 收集所有结果
    for group_result in group_results:
        to_eval_list.extend(group_result)
    
    return to_eval_list

In [ ]:
loop = asyncio.get_event_loop()
# 运行异步函数
eval_result_list = loop.run_until_complete(run_parallel_processing(10000,6))

An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from HvQ2mD2gwaFZ7jXR93PImmP1SDyl4tcJXJeH with message "Connection reset by peer for the huolala-distil-7b-920-full-rag-all-thinking-epoch-2-endpoint endpoint. Please retry.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/huolala-distil-7b-920-full-rag-all-thinking-epoch-2-endpoint in account 434444145045 for more information.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 loop = asyncio.get_event_loop()                                                              │
│   2 # 运行异步函数                                                                               │
│ ❱ 3 eval_result_list = loop.run_until_complete(run_parallel_processing(10000,6))                 │
│   4                                                                                              │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/nest_asy │
│ ncio.py:92 in run_until_complete                                                                 │
│                                                                                                  │
│    89 │   │   │   if f is not future:                                                            │
│    90 │   │   │   │   f._log_destroy_pending = False                                             │
│    91 │   │   │   while not f.done():                                                            │
│ ❱  92 │   │   │   │   self._run_once()                                                           │
│    93 │   │   │   │   if self._stopping:                                                         │
│    94 │   │   │   │   │   break                                                                  │
│    95 │   │   │   if not f.done():                                                               │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/nest_asy │
│ ncio.py:115 in _run_once                                                                         │
│                                                                                                  │
│   112 │   │   │   else min(max(                                                                  │
│   113 │   │   │   │   scheduled[0]._when - self.time(), 0), 86400) if scheduled                  │
│   114 │   │   │   else None)                                                                     │
│ ❱ 115 │   │   event_list = self._selector.select(timeout)                                        │
│   116 │   │   self._process_events(event_list)                                                   │
│   117 │   │                                                                                      │
│   118 │   │   end_time = self.time() + self._clock_resolution                                    │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/selectors.py:468 in    │
│ select                                                                                           │
│                                                                                                  │
│   465 │   │   │                                                                                  │
│   466 │   │   │   ready = []                                                                     │
│   467 │   │   │   try:                                                                           │
│ ❱ 468 │   │   │   │   fd_event_list = self._selector.poll(timeout, max_ev)                       │
│   469 │   │   │   except InterruptedError:                                                       │
│   470 │   │   │   │   return ready                                                               │
│   471 │   │   │   for fd, event in fd_event_list:                

In [ ]:

# t1 = time.time()
# i = 0
# while True:
#     predictor.predict(payload) 
#     i += 1
#     if i % 50 == 0:
#         print(response)
#         print(f"---{i}----time: {(time.time() - t1):.1f}")
#     if i == 1000000:
#         break

### 4.4 Optionally clean up the environment

- Deregister scalable target
- Delete cloudwatch alarms
- Delete scaling policies


In [68]:
try:
    # Deregister the scalable target for AAS
    aas_client.deregister_scalable_target(
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension=scalable_dimension,
    )
    print(f"Scalable target for [b]{resource_id}[/b] deregistered. ✅")
except aas_client.exceptions.ObjectNotFoundException:
    print(f"Scalable target for [b]{resource_id}[/b] not found!.")

print("---" * 10)

# Delete CloudWatch alarms created for Step scaling policy
try:
    cloudwatch_client.delete_alarms(AlarmNames=[step_scaling_alarm_name])
    print(f"Deleted CloudWatch step scaling scale-out alarm [b]{step_scaling_alarm_name} ✅")
except cloudwatch_client.exceptions.ResourceNotFoundException:
    print(f"CloudWatch scale-out alarm [b]{step_scaling_alarm_name}[/b] not found.")


# Delete step scaling policies
print("---" * 10)

try:
    aas_client.delete_scaling_policy(
        PolicyName=step_scaling_policy_name,
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    )
    print(f"Deleted scaling policy [i green]{step_scaling_policy_name} ✅")
except aas_client.exceptions.ObjectNotFoundException:
    print(f"Scaling policy [i]{step_scaling_policy_name}[/i] not found.")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 try:                                                                                        │
│    2 │   # Deregister the scalable target for AAS                                                │
│ ❱  3 │   aas_client.deregister_scalable_target(                                                  │
│    4 │   │   ServiceNamespace="sagemaker",                                                       │
│    5 │   │   ResourceId=resource_id,                                                             │
│    6 │   │   ScalableDimension=scalable_dimension,                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'aas_client' is not defined

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   │   ScalableDimension=scalable_dimension,                                               │
│    7 │   )                                                                                       │
│    8 │   print(f"Scalable target for [b]{resource_id}[/b] deregistered. ✅")                     │
│ ❱  9 except aas_client.exceptions.ObjectNotFoundException:                                       │
│   10 │   print(f"Scalable target for [b]{resource_id}[/b] not found!.")                          │
│   11                                                                                             │
│   12 print("---" * 10)                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'aas_client' is not defined



- Delete inference component
- Delete endpoint
- delete endpoint-config



In [ ]:
sm_client.delete_inference_component(InferenceComponentName=inference_component_name)
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

In [69]:
!aws sagemaker delete-inference-component --inference-component-name {inference_component_name}


In [70]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}


In [71]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}


In [72]:
!aws sagemaker delete-model --model-name {model_name}

In [29]:
# !pip install boto3
import re
import json
import boto3
from botocore.exceptions import ClientError
import time
max_retries = 8
retry_delay = 10

# 更改成model hub部署的endpoint名称和region name
# endpoint_name = "Qwen2-5-VL-3B-Instruct-2025-03-07-15-34-sglang-endpoint"
# inference_component_name = "Qwen2-5-VL-3B-Instruct-2025-03-07-15-34-sglang-component"
endpoint_name = "Qwen2-5-VL-3B-Instruct-2025-03-08-05-30-sglang-endpoint"
inference_component_name = "Qwen2-5-VL-3B-Instruct-2025-03-08-05-30-sglang-component"


region_name = 'us-east-1'
runtime = boto3.client('runtime.sagemaker',region_name=region_name)
payload = {
    "messages": [
    {
        "role": "user",
        "content": "who are you"
    }
    ],
    "max_tokens": 1024,
    "stream": False,
    "model":"any"
}

print("# 指数回退重试 --------------")
def invoke_with_exponential_backoff(endpoint_name, payload,retry_delay):
    for attempt in range(max_retries):
        try:
            response = runtime.invoke_endpoint(
                EndpointName=endpoint_name,
                InferenceComponentName=inference_component_name,
                ContentType='application/json',
                Body=json.dumps(payload)
            )
            return response
        except ClientError as e:
            if e.response['Error']['Code'] == 'ValidationError' and 'Inference Component has no capacity' in e.response['Error']['Message']:
                if attempt < max_retries - 1:
                    print(f"Inference Component has no capacity, retrying in {retry_delay} seconds (attempt {attempt+1}/{max_retries})")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # Exponential backoff
                else:
                    print("Maximum retries reached, unable to process request")
            else:
                raise
            
response = invoke_with_exponential_backoff(endpoint_name, payload,retry_delay)
print(json.loads(response['Body'].read())["choices"][0]["message"]["content"])


print("# 非流式 --------------")
# 非流式
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["message"]["content"])


payload = {
    "messages": [
    {
        "role": "user",
        "content": "Write a quick sort in python"
    }
    ],
    "max_tokens": 1024,
    "stream": True,
    "model":"any"
}


# 流式
print("# 流式 --------------")
response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            data = json.loads(match.group(1).strip())
            last_idx = match.span()[1]
            print(data["choices"][0]["delta"]["content"], end="",flush=True)
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]

# 指数回退重试 --------------


I am a large language model created by Alibaba Cloud. I am called Qwen and I was trained to generate human-like responses to a wide range of questions and prompts.
# 非流式 --------------
I am a large language model, created by OpenAI.
# 流式 --------------
Certainly! Quick sort is a popular and efficient sorting algorithm that uses the divide-and-conquer strategy. Below is a Python implementation of the quick sort algorithm:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

# Example usage:
arr = [3, 6, 8, 10, 1, 2, 1]
print("Original array:", arr)
sorted_arr = quick_sort(arr)
print("Sorted array:", sorted_arr)
```

### Explanation:
1. **Base Case**: If the array has 1 or 0 elements, it is already sorted.
2. **Pivot Selection